### Collisional-radiative model for Argon Plasma
#### Kinetics loading
See: https://doi.org/10.1063/1.3585688

In [1]:
import os
import json
import numpy as np

from silx.io.dictdump import dicttoh5

Inputs

In [2]:
path_to_species = "./../species"
path_to_dtb = "/home/zanardi/Codes/ML/RONEK/run/Argon/argon_cr/data"
Tmin = 200

In [3]:
# Save path
out_path = "./../kinetics/"
os.makedirs(out_path, exist_ok=True)

Read species

In [4]:
species = {}
for filename in os.listdir(path_to_species):
  filename = path_to_species + "/" + filename
  with open(filename, "r") as file:
    ispecies = json.load(file)
  species[ispecies["name"]] = ispecies

Read temperature grid

In [5]:
Tall = np.loadtxt(path_to_dtb+"/T", skiprows=4)
indices = np.where(Tall>Tmin)[0]
T = Tall[indices]

Dimensions

In [6]:
nb_pts = len(indices)
nb_lev_Ar = species["Ar"]["nb_lev"]
nb_lev_Arp = species["Ar+"]["nb_lev"]

Read rates

In [7]:
rates = {"T": T}

**Excitation**

- Electron impact excitation:
$$\quad \text{Ar}(i) + \text{e}^- \xrightleftharpoons[F_{ji}]{C_{ij}} \text{Ar}(j) + \text{e}^-; \quad j > i$$
- Heavy-particle impact excitation:
$$\quad \text{Ar}(i) + \text{Ar}(1) \xrightleftharpoons[L_{ji}]{K_{ij}} \text{Ar}(j) + \text{Ar}(1); \quad j > i$$

In [8]:
def read_exc(name):
  X = np.zeros((nb_lev_Ar,1,nb_lev_Ar,1,nb_pts))
  for i in range(nb_lev_Ar):
    for j in range(i+1,nb_lev_Ar):
      X[i,0,j,0] = np.loadtxt(path_to_dtb+f"/{name}{i}_{j}")[indices]
  return X

In [9]:
for name in ("C", "K"):
  rates[name] = read_exc(name)
# Correction factor applied to the rate from the ground state (i=0)
# See: M. G. Kapper's PhD thesis (2009)
rates["K"][0,0,:,0] *= 0.085

**Ionization**

- Electron impact ionization:
$$\quad \text{Ar}(i) + \text{e}^- \xrightleftharpoons[O_{ji}]{S_{ij}} \text{Ar}^+(j) + \text{e}^- + \text{e}^-$$
- Heavy-particle impact ionization:
$$\quad \text{Ar}(i) + \text{Ar}(1) \xrightleftharpoons[W_{ji}]{V_{ij}} \text{Ar}^+(j) + \text{e}^- + \text{Ar}(1)$$

In [10]:
def read_ion(name):
  X = np.zeros((nb_lev_Ar,1,nb_lev_Arp,1,1,nb_pts))
  for i in range(nb_lev_Ar):
    found = False
    jc_i = species["Ar"]["lev"]["jc"][i]
    for j in range(nb_lev_Arp):
      jc_j = species["Ar+"]["lev"]["jc"][j]
      # Ar+ and Ar_i must have the same value for the core angular momentum
      if (jc_i == jc_j):
        found = True
        X[i,0,j,0,0] = np.loadtxt(path_to_dtb+f"/{name}{i}")[indices]
        break
    if (not found):
      raise ValueError("Ionization rate not found! Check 'jc' values.")
  return X

In [11]:
for name in ("S", "V"):
  rates[name] = read_ion(name)

**Radiation**

Radiation absorption is taken into account via the use of escape $\Lambda_{ij}$ factors that can take values between 0 (optically thick plasma) and 1 (optically thin plasma). An optically thin plasma is considered here ($\Lambda_{ij}=1$).
- Spontaneous emission (bound-bound): $$\quad \text{Ar}(i) + h\nu_{ij} \xrightleftharpoons[\Lambda_{ij}A_{ij}]{(1-\Lambda_{ij})A_{ij}} \text{Ar}(j)$$

In [12]:
rates["A"] = np.loadtxt(path_to_dtb+"/A")

- Photo-ionization (bound-free and free-bound):
$$\quad \text{Ar}(i) + h\nu_{ij} \xrightleftharpoons[\Lambda_{ij}R_{ij}]{(1-\Lambda_{ij})R_{ij}} \text{Ar}^+(j) + \text{e}^-$$

In [13]:
for name in ("R", "Rp"):
  rates[name] = read_ion(name)

**First moments**
- Electron-neutral collision rate

In [14]:
rates["ken"] = np.loadtxt(path_to_dtb+"/ken")[indices]

Save raw rates

In [15]:
dicttoh5(rates, out_path + "/rates_raw.hdf5")